In [21]:
# Important for reloading modified files
%load_ext autoreload
%autoreload 2

import numpy as np
from modules import Network, LinearLayer, Sigmoid, ReLU, MSE, CrossEntropyLoss

import codecs, json 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Forward Pass

In this exercise you will implement function evaluations for some of the most fundamental building blocks of modern arificial neural networks. The idea is that you create a network consisting of several layers each of which implements the `forward` function inherited from the base class `Module`. A skeleton for your implementations is provided by `modules.py`. Work through this notebook to validate your code.

## Network

As mentioned in the lecture, the notion of a *layer* is not well-defined and we may even regard a whole network as a layer predicting a desired output from input data. Therefore we want our class ```Network``` to be a subclass of ```Module```. Of course, we need a mechanism for implementing this high-level view of mapping input to output. This is achieved by sequential execution of certain network layers, which we want to store in our ```Network``` class. Follow the comments below and complete the code. You can test your implementation at the end of this notebook after finishing the other tasks.

## Layers

As mentioned above, layers are the building blocks of neural networks. In this section, you will familiarize yourself with some of the most common types of layers in artificial neural networks.  

### Linear Layer

Linear layers are a simple and yet extremely powerful tool for the design of deep neural networks. This is partly due to the highly parallelized and thus efficient computation of linear transformations on modern GPUs. Implement the class `LinearLayer` and test you implementation by running the cell below:

In [13]:
W = np.ones((3, 4))
b = np.linspace(1, 3, 3)

ll = LinearLayer(W, b)
x = np.ones(4)

assert np.abs(np.max(ll.forward(x) - [5, 6, 7])) < 1e-6

In [14]:
#relu check
W = np.random.random((3, 4))
W[0,0] = -1
print(np.maximum(W,0)) #consider 2nd arg as a value
print(np.max(W,0)) #2nd arg as axis

[[0.         0.65072547 0.77406075 0.81094198]
 [0.33880147 0.50294339 0.44280834 0.83434484]
 [0.91641693 0.31548472 0.88313714 0.74547707]]
[0.91641693 0.65072547 0.88313714 0.83434484]


### Sigmoid

The sigmoid activation has been of great importance in the field of machine learning ever since. Biologically motivated at first, it carries out the mapping $x \mapsto {e^x} / (1 + e^x)$. Complete the forward pass of the class `Sigmoid`:

In [15]:
assert np.abs(np.max(Sigmoid().forward([0, -1, 10]) - [0.5, 0.2689414, 1.0])) < 1e-6

### ReLU

Gaining tremendous success in the last couple of years, **Re**ctified __L__inear **U**nits are now the most common activations in use. Their power comes from the simplicity of their forward pass, which discards all negative values, i.e. $x \mapsto \max(x, 0)$, where the maximum is to be understood element-wise. 

Test your implementation:

In [16]:
assert np.abs(np.max(ReLU().forward([-3.14, 0, 1, 10]) - [0., 0., 1., 10])) < 1e-6

## Loss

The task of a network is predicting a desired output from input data. The quality of the prediction is assessed by *loss functions* comparing the predicted output with the target or groundtruth. In our implementation we can model loss as subclass of Module. It therefore also features a forward function. However, we now require it to take an argument for the output of the network **and** an argument for the target.

### MSE

We already discussed the MSE loss in the context of linear regression. Implement the `forward` function calculating the mean squared difference of prediction and target.

Test your implementation:

In [17]:
assert np.abs(np.max(MSE().forward(np.array([0., 1., 2., 1.5]), np.array([0., 1., 1., -1.])) - 7.25/4)) < 1e-6

### Cross Entropy

Another important task in the regime of machine learning is *classification*. Given some input $x$ we want to predict a discrete class label $l\in\{1, \ldots, L\}$. In order to train neural networks, we need a differentiable forward pass, which renders a discrete prediction function impossible. Therefore, we want to predict a vector in $\mathbb{R}^L$ representing our believe for each label. We can actually transform this vector into a valid probability distribution using the softmax function (https://en.wikipedia.org/wiki/Softmax_function)
$$
\sigma \, \colon \, \mathbb{R}^L \to \left\{ \sigma \in \mathbb{R}^L \, \middle| \, \sigma_i > 0, \sum_{i=1}^L \sigma_i = 1 \right\}, \, \sigma_j ( z ) = \frac{e^{z_j}}{\sum_{i=1}^L e^{z_i}} \text{ for $j \in \left\{ 1, \ldots, L \right\}$}.
$$

This in turn allows us to define a proper loss function: we simply take the negative log of the predicted probability of the target label $l$, i.e. $\ell (x, l) = -\log (\sigma_l (x))$. Implement the cross entropy loss, where $x$ is the prediction of our network and $l$ is given by the target label.

Test your implementation:

In [18]:
assert np.abs(np.max(CrossEntropyLoss().forward(np.array([-3.14, 0, 1, 10]), 0) - 13.1401)) < 1e-4

## Final test of the network implementation

In [19]:
W1 = np.ones((3, 4))
b1 = np.linspace(1, 3, 3)
ll1 = LinearLayer(W1, b1)

W2 = np.ones((1, 3))
b2 = np.ones((1))
ll2 = LinearLayer(W2, b2)

relu = ReLU()

net = Network([ll1, relu])

net.add_layer(ll2)

assert np.abs(np.max(net.forward(np.array([-3.14, 0, 1, 10]).T) - 30.58)) < 1e-4